# Reading DAGs

In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [0]:
spark = (
    SparkSession
    .builder
    .config("spark.driver.memory", "10g")
    .config("spark.sql.files.maxPartitionBytes", "268435456")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .master("local[*]")
    .getOrCreate()
)
sc = spark.sparkContext

<h1> Topics </h1>

1. Reading Files (parquet)
2. Narrow Operations
   - `filter`
   - `withColumn`: adding/modifying a column
   - `select`: selecting relevant column
3. Wide Operations
   - Joins
     - Sort Merge Join
     - Broadcast Join
   - GroupBy
     - `count`
     - `sum`
     - `countDistinct`

# Reading File

In [0]:
transactions_file = "/mnt/data/data/data_skew/transactions.parquet"
df_transactions = spark.read.parquet(transactions_file)

In [0]:
df_transactions.rdd.getNumPartitions()

Out[4]: 6

In [0]:
sc.setJobDescription("Show Action")
df_transactions.show(5, False)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt   |city       |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9S03OQJ|2018-10-07|2018|10   |7  |Entertainment|10.42 |boston     |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYIAPPNU066CJ5R|2016-03-27|2016|3    |27 |Motor/Travel |44.34 |portland   |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TETSXIK4BLXHJ6W|2011-04-11|2011|4    |11 |Entertainment|3.18  |chicago    |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TQKL1QFJY3EM8LO|2018-02-22|2018|2    |22 |Groceries    |268.97|los_angeles|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYL6DFP09PPXMVB|2010-10-16|2010|10   |16 |Entertainment|2.66  |chicago    |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
only showi

## Switch to Spark UI 
> **Note** A `batch` refers to a group of rows that are processed together.

In [0]:
customers_file = "/mnt/data/data/data_skew/customers.parquet"
df_customers = spark.read.parquet(customers_file)

In [0]:
df_customers.show(5, False)

+----------+-------------+---+------+----------+-----+-----------+
|cust_id   |name         |age|gender|birthday  |zip  |city       |
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9|Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |
|C00B971T1J|Aaron Austin |37 |Female|12/16/2004|30332|chicago    |
|C00WRSJF1Q|Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |
|C01AZWQMF3|Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|
|C01BKUFRHA|Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows



# Narrow Transformations
- `filter` rows where `city='boston'`
- `add` a new column: adding `first_name` and `last_name`
- `alter` an exisitng column: adding 5 to `age` column
- `select` relevant columns

In [0]:
df_narrow_transform = (
    df_customers
    .filter(F.col("city") == "boston")
    .withColumn("first_name", F.split("name", " ").getItem(0))
    .withColumn("last_name", F.split("name", " ").getItem(1))
    .withColumn("age", F.col("age") + F.lit(5))
    .select("cust_id", "first_name", "last_name", "age", "gender", "birthday")
)

df_narrow_transform.write.format("noop").mode("overwrite").save("../data/test/df_narrow_transform.parquet")

In [0]:
df_narrow_transform.show(7, False)

+----------+----------+---------+----+------+---------+
|cust_id   |first_name|last_name|age |gender|birthday |
+----------+----------+---------+----+------+---------+
|C007YEYTX9|Aaron     |Abbott   |39.0|Female|7/13/1991|
|C08XAQUY73|Aaron     |Lambert  |59.0|Female|11/5/1966|
|C094P1VXF9|Aaron     |Lindsey  |29.0|Male  |9/21/1990|
|C097SHE1EF|Aaron     |Lopez    |27.0|Female|4/18/2001|
|C0DTC6436T|Aaron     |Schwartz |57.0|Female|7/9/1962 |
|C0R42FPHRH|Abbie     |Reyes    |68.0|Male  |10/8/1995|
|C0RZV4BH7T|Abbie     |Stevenson|41.0|Male  |2/10/1971|
+----------+----------+---------+----+------+---------+
only showing top 7 rows



In [0]:
df_customer_gt_50 = (
    df_customers
    .filter(F.col("age").cast("int") > 50)
)
df_customer_gt_50.write.format("noop").mode("overwrite").save("../data/test/df_customer_gt_50.parquet")

# Wide Transformations
1. Joins
   - Sort Merge Join
   - Broadcast Join
2. GroupBy
   - `count`
   - `countDistinct`
   - `sum`

## 1. Joins

### Sort Merge Join

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [0]:
df_joined = (
    df_transactions.join(
        df_customers,
        how="inner",
        on="cust_id"
    )
)

In [0]:
df_joined.write.format("noop").mode("overwrite").save("../data/test/df_joined.parquet")

### Broadcast Join

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10485760)

In [0]:
df_broadcast_joined = (
    df_transactions.join(
        F.broadcast(df_customers),
        how="inner",
        on="cust_id"
    )
)

In [0]:
df_broadcast_joined.write.format("noop").mode("overwrite").save("../data/test/df_broadcast_joined.parquet")

## 2. GroupBy

### GroupBy Count

In [0]:
df_city_counts = (
    df_transactions
    .groupBy("city")
    .count()
)

In [0]:
df_city_counts.show(5, False)

+---------+-------+
|city     |count  |
+---------+-------+
|san_diego|3977780|
|chicago  |3979023|
|denver   |3980274|
|boston   |3978268|
|seattle  |3980022|
+---------+-------+
only showing top 5 rows



In [0]:
df_txn_amt_city = (
    df_transactions
    .groupBy("city")
    .agg(F.sum("amt").alias("txn_amt"))
)

In [0]:
df_txn_amt_city.show(5, False)

+---------+--------------------+
|city     |txn_amt             |
+---------+--------------------+
|san_diego|3.297982686000053E8 |
|chicago  |3.2988120044000757E8|
|denver   |3.2988149564000154E8|
|boston   |3.3010095633000124E8|
|seattle  |3.3019513776999843E8|
+---------+--------------------+
only showing top 5 rows



### GroupBy Count Distinct 

In [0]:
df_txn_per_city = (
    df_transactions
    .groupBy("city")
    .agg(F.countDistinct("txn_id").alias("txn_count"))
)

In [0]:
df_txn_per_city.show(5, False)

+---------+---------+
|city     |txn_count|
+---------+---------+
|san_diego|3977780  |
|chicago  |3979023  |
|denver   |3980274  |
|boston   |3978268  |
|seattle  |3980022  |
+---------+---------+
only showing top 5 rows



In [0]:
spark.stop()